# Import Module

In [1]:
import os
import torch
import numpy as np
import nibabel as nib
from scipy import ndimage
#from models import DP_resUNet#, DP_UNet
from models_atten import DP_atten_resUNet
from inference import sliding_inference, tprocess#, inference_and_output
#from data_preprocess_and_loader import Dataset_out

tbatch_size = 1
test_image_path = 'path that save the test data'
test_save_folder = 'prediction_site'
test_site = 'name_test_site'

base_path = 'directory to save model'
model_folder = 'model_folder_name'

mpath = os.path.join(base_path, model_folder)
model_name = 'last.tar' #model_name

os.makedirs(os.path.join(mpath,test_save_folder), exist_ok=True)

tlabel = []
tpath = []
tpatients = []

inplane_size = 256
slice_num = 64
if 'bi' in model_folder:
    biparametric = True
    img_channels = 2
elif 'single' in model_folder:
    biparametric = False
    img_channels = 1

if torch.cuda.is_available():
    model = DP_atten_resUNet(img_channels = img_channels, n_classes = 1).cuda()
    #model = torch.nn.DataParallel(model)
own_state = torch.load(os.path.join(base_path, model_folder, model_name))
model.load_state_dict(own_state)

k = os.listdir(test_image_path)
for j in k:
    tpath.append(test_image_path + '/' + j)    
    tpatients.append(j)

In [2]:
test_dice = []
test_sen = []
test_pre = []

for ind in range(len(tpath)):
    img, gt = tprocess(tpath[ind], biparametric)
    val = sliding_inference(img, model, inplane_size, slice_num)
    
    gt[np.where(np.isnan(gt))] = 0
    val[np.where(np.isnan(val))] = 0
    val[val>=0.5] = 1
    val[val<0.5] = 0
    
    for s in range(val.shape[-1]):
        val[:,:,s] = ndimage.binary_fill_holes(val[:,:,s])
        
    intersection = intersection = np.sum(gt * val)
    dice = (2. * intersection + 1e-8) / (np.sum(gt) + np.sum(val) + 1e-8)
    sen = (intersection + 1e-8) / (np.sum(gt) + 1e-8)
    pre = (intersection + 1e-8) / (np.sum(val) + 1e-8)
    print(tpath[ind], dice)
    test_sen.append(sen)    
    test_dice.append(dice)
    test_pre.append(pre)
    
    if 'AN' in tpath[ind]:
        tumor_type = 'AN'        
    if 'Men' in tpath[ind]:
        tumor_type = 'Mena'
    if 'Met' in tpath[ind]:
        tumor_type = 'Mets'
    tmp_pid = tpath[ind].split('/')[-1]
    pid = tmp_pid[:-4]
    
    if tumor_type == 'AN':
        nii_mimg_path = os.path.join('D:/M4/ZZc' , pid[3:], 'axc', 'mimg.nii')
    elif tumor_type == 'Mena':        
        nii_mimg_path = os.path.join('F:/Meningioma/20210402' , pid[5:], 'axc', 'mimg.nii')
    elif tumor_type == 'Mets':   
        nii_mimg_path = os.path.join('E:/Metastasis/20211128' , pid[5:], 'axc', 'mimg.nii')
    nii_mimg = nib.load(nii_mimg_path).get_fdata()
    
    affine = nib.load(nii_mimg_path).affine
    imgp = nib.Nifti1Image(np.rot90(val[:,:,0:nii_mimg.shape[2]], 3), affine)
    nib.save(imgp,os.path.join('build',mpath,test_save_folder,pid +'.nii.gz'))

print(np.median(test_dice))
print(np.median(test_sen))
print(np.median(test_pre))

import pandas as pd    
df = pd.DataFrame({'path': tpatients,
                   'dice': test_dice,
                   'sensitivity': test_sen,
                   'precision': test_pre,
                  })
writer = pd.ExcelWriter(os.path.join(mpath, 'Test_dice_'+test_site+'.xlsx'))
df.to_excel(writer,'Sheet1',index=False)
writer.close()

F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1305833010.npz 0.9007368877334169
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1305845000.npz 0.8499427262331049
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1305847000.npz 0.9110240626749294
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1305870000.npz 0.9072273324575368
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1405893010.npz 0.9028571428574512
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1405912010.npz 0.9122685983673539
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1405926010.npz 0.8612440191400839
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1405932000.npz 0.9085457271378029
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1405936000.npz 0.8856447688578388
F:/20220309_process4FL/TVGH/

F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1606874000.npz 0.9301036235232228
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1606886010.npz 0.9305671392827718
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1706913000.npz 0.9188311688313885
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1706957000.npz 0.8782489740098735
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1706969010.npz 0.9128205128206784
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1706975000.npz 0.9611085355174568
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1707031000.npz 0.9013202058628299
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1707040010.npz 0.9041626331079179
F:/20220309_process4FL/TVGH/brain_tumor_nomask_biparametric_twcc4/test/AN_1707041000.npz 0.9332050871006378
F:/20220309_process4FL/TVGH/